In [1]:
import os
import pickle 
import random
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import torch

import sys
sys.path.insert(0, '..')
from src.data import TimeSeries
from src.methods import rl
from src.environment import TimeSeriesEnv

In [2]:
# GLOBALS
SEED = 3141
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
for dir in os.listdir('../data'):
    if dir in ['nitrogen-generator','wrapper-machine']:
        continue

    print(dir.upper())
    # Data loading
    data_dir_path = os.path.join('../data', dir)
    
    X_train = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_train.pkl'), 'rb'))
    X_valid = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_valid.pkl'), 'rb'))
    X_test = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_test.pkl'), 'rb'))
    y_train = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_train.pkl'), 'rb'))
    y_valid = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_valid.pkl'), 'rb'))
    y_test = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_test.pkl'), 'rb'))
    

    # Data loading
    data_dir_path = os.path.join('../data', dir)
    if dir == 'nitrogen-generator':
        tf = '%Y-%m-%d %H:%M:%S'
    else:
        tf = '%Y-%m-%d %H:%M:%S.%f'

    train_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'train.csv')
    )
    valid_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'val.csv')
    )
    test_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'test.csv')
    )

    # Data prep
    train_ts.parse_datetime('timestamp', tf)
    valid_ts.parse_datetime('timestamp', tf)
    test_ts.parse_datetime('timestamp', tf)

    train_ts.split_by_day()
    valid_ts.split_by_day()
    test_ts.split_by_day()

    temp = pd.concat(
        train_ts.time_series[k].drop(
            columns=['timestamp','PW_0.5h','date','time']
        ) for k in train_ts.time_series.keys()
    )

    FEATURE_COLS = [
        c for c in temp.columns if np.std(temp[c])!=0
    ]
    LABEL_COL = 'PW_0.5h'

    temp = None
    del temp

    # Make environments
    train_env = TimeSeriesEnv(train_ts, FEATURE_COLS, LABEL_COL, True)
    valid_env = TimeSeriesEnv(valid_ts, FEATURE_COLS, LABEL_COL, False)
    test_env = TimeSeriesEnv(test_ts, FEATURE_COLS, LABEL_COL, False)

    # Model prep
    hid_dim = 0
    for i in range(10):
        if 2**i > len(FEATURE_COLS):
            hid_dim = 2**(i+1)
            break
    print('Features: ', len(FEATURE_COLS))
    print('Hidden Dim: ',hid_dim)
    alpha_stop = rl.NeuralNetGuidedMCTS(
        in_dim=len(FEATURE_COLS),
        hid_dim=hid_dim,
        save_dir=os.path.join('../results/rl',dir),
        n_actions=2,
        n_sim=100,
        lr=1e-4,
        weight_decay=0.01,
        gamma=0.999,
        bsz=64,
        device='cpu'
    )

    # Train
    epochs = 100
    train_actions, train_rewards = alpha_stop.train(epochs, train_env, valid_env)

    # Load and test best model
    alpha_stop.net.load_state_dict(
        torch.load(
            os.path.join('../results/rl',dir,'network.pt')
        )
    )
    alpha_stop.mcts = pickle.load(open(os.path.join('../results/rl',dir,'mcts.pkl'),'rb'))
    test_actions, test_rewards = alpha_stop.run(test_env)
    
    # Save output
    pickle.dump(train_actions, open(os.path.join('../results/rl',dir,'train_actions.pkl'), 'wb'))
    pickle.dump(train_rewards, open(os.path.join('../results/rl',dir,'train_rewards.pkl'), 'wb'))
    pickle.dump(test_actions, open(os.path.join('../results/rl',dir,'test_actions.pkl'), 'wb'))
    pickle.dump(test_rewards, open(os.path.join('../results/rl',dir,'test_rewards.pkl'), 'wb'))

print("<< PROCESS COMPELTE >>")

In [ ]:
for dir in os.listdir('../data'):
    if dir in ['nitrogen-generator','wrapper-machine']:
        continue

    print(dir.upper())
    # Data loading
    data_dir_path = os.path.join('../data', dir)
    
    X_train = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_train.pkl'), 'rb'))
    X_valid = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_valid.pkl'), 'rb'))
    X_test = pickle.load(open(os.path.join('../data',dir,'preprocessed/X_test.pkl'), 'rb'))
    y_train = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_train.pkl'), 'rb'))
    y_valid = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_valid.pkl'), 'rb'))
    y_test = pickle.load(open(os.path.join('../data',dir,'preprocessed/y_test.pkl'), 'rb'))
    

    # Data loading
    data_dir_path = os.path.join('../data', dir)
    if dir == 'nitrogen-generator':
        tf = '%Y-%m-%d %H:%M:%S'
    else:
        tf = '%Y-%m-%d %H:%M:%S.%f'

    train_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'train.csv')
    )
    valid_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'val.csv')
    )
    test_ts = TimeSeries.from_csv(
        'pandas',
        os.path.join(data_dir_path, 'test.csv')
    )

    # Data prep
    train_ts.parse_datetime('timestamp', tf)
    valid_ts.parse_datetime('timestamp', tf)
    test_ts.parse_datetime('timestamp', tf)

    train_ts.split_by_day()
    valid_ts.split_by_day()
    test_ts.split_by_day()

    temp = pd.concat(
        train_ts.time_series[k].drop(
            columns=['timestamp','PW_0.5h','date','time']
        ) for k in train_ts.time_series.keys()
    )

    FEATURE_COLS = [
        c for c in temp.columns if np.std(temp[c])!=0
    ]
    LABEL_COL = 'PW_0.5h'

    temp = None
    del temp

    # Make environments
    train_env = TimeSeriesEnv(train_ts, FEATURE_COLS, LABEL_COL, True)
    valid_env = TimeSeriesEnv(valid_ts, FEATURE_COLS, LABEL_COL, False)
    test_env = TimeSeriesEnv(test_ts, FEATURE_COLS, LABEL_COL, False)

    # Model prep
    hid_dim = 0
    for i in range(10):
        if 2**i > len(FEATURE_COLS):
            hid_dim = 2**(i+1)
            break
    print('Features: ', len(FEATURE_COLS))
    print('Hidden Dim: ',hid_dim)
    ppo = rl.PPOAgent(
        in_dim=len(FEATURE_COLS),
        hid_dim=hid_dim,
        clip=0.2,
        update_freq=10,
        save_dir=os.path.join('../results/rl',dir),
        lr=3e-4,
        gamma=0.99,
        bsz=64,
        device='cuda'
    )

    # Train
    epochs = 1000
    train_actions, train_rewards = ppo.train(epochs, train_env, valid_env)

    # Load and test best model
    ppo.policy_net.load_state_dict(
        torch.load(
            os.path.join('../results/rl',dir,'ppo-policy-network.pt')
        )
    )
    test_actions, test_rewards = ppo.run(test_env)
    
    # Save output
    pickle.dump(train_actions, open(os.path.join('../results/rl',dir,'ppo_train_actions.pkl'), 'wb'))
    pickle.dump(train_rewards, open(os.path.join('../results/rl',dir,'ppo_train_rewards.pkl'), 'wb'))
    pickle.dump(test_actions, open(os.path.join('../results/rl',dir,'ppo_test_actions.pkl'), 'wb'))
    pickle.dump(test_rewards, open(os.path.join('../results/rl',dir,'ppo_test_rewards.pkl'), 'wb'))

print("<< PROCESS COMPELTE >>")

BLOOD-REFRIGERATOR
Features:  12
Hidden Dim:  32


Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/60166 [00:00<?, ?it/s]

Date:   2022-11-07  stopped at:  0
Date:   2022-11-18  stopped at:  0
Date:   2022-11-08  stopped at:  0
Date:   2022-11-11  stopped at:  0
Date:   2022-11-19  stopped at:  0
Date:   2022-11-15  stopped at:  0
Date:   2022-11-12  stopped at:  0
Date:   2022-11-03  stopped at:  1
Date:   2022-11-06  stopped at:  0
Date:   2022-11-24  stopped at:  2
Date:   2022-10-31  stopped at:  5
Date:   2022-11-02  stopped at:  0
Date:   2022-11-10  stopped at:  1
Date:   2022-11-09  stopped at:  1
Date:   2022-11-13  stopped at:  0
Date:   2022-11-20  stopped at:  5
Date:   2022-11-17  stopped at:  4
Date:   2022-11-21  stopped at:  10
Date:   2022-11-14  stopped at:  0
Date:   2022-11-01  stopped at:  0
Date:   2022-11-23  stopped at:  0
Date:   2022-11-05  stopped at:  1
Date:   2022-11-22  stopped at:  1
Date:   2022-11-04  stopped at:  3
Date:   2022-11-16  stopped at:  1
Date:   2022-11-24  stopped at:  4
Date:   2022-11-25  stopped at:  1
Date:   2022-11-26  stopped at:  1
Date:   2022-11-27 

  0%|          | 0/60166 [00:00<?, ?it/s]

...

Date:   2022-11-22  stopped at:  15
Date:   2022-11-05  stopped at:  10
Date:   2022-11-20  stopped at:  53
Date:   2022-11-14  stopped at:  11
Date:   2022-11-02  stopped at:  10
Date:   2022-11-24  stopped at:  7
Date:   2022-11-03  stopped at:  21
Date:   2022-11-18  stopped at:  7
Date:   2022-11-15  stopped at:  2
Date:   2022-10-31  stopped at:  23
Date:   2022-11-23  stopped at:  9
Date:   2022-11-10  stopped at:  1
Date:   2022-11-07  stopped at:  1
Date:   2022-11-09  stopped at:  6
Date:   2022-11-11  stopped at:  1337
Date:   2022-11-01  stopped at:  1
Date:   2022-11-17  stopped at:  78
Date:   2022-11-08  stopped at:  41
Date:   2022-11-06  stopped at:  20
Date:   2022-11-21  stopped at:  3
Date:   2022-11-19  stopped at:  62
Date:   2022-11-13  stopped at:  57
Date:   2022-11-04  stopped at:  9
Date:   2022-11-16  stopped at:  295
Date:   2022-11-12  stopped at:  4
Date:   2022-11-24  stopped at:  0
Date:   2022-11-25  stopped at:  39
Date:   2022-11-26  stopped at:  1
Da

  0%|          | 0/60166 [00:00<?, ?it/s]

Date:   2022-11-05  stopped at:  4
Date:   2022-11-16  stopped at:  376
Date:   2022-11-14  stopped at:  96
Date:   2022-11-12  stopped at:  25
Date:   2022-11-11  stopped at:  432
Date:   2022-11-10  stopped at:  0
Date:   2022-11-03  stopped at:  18
Date:   2022-11-06  stopped at:  27
Date:   2022-11-13  stopped at:  36
Date:   2022-11-08  stopped at:  12
Date:   2022-11-23  stopped at:  31
Date:   2022-11-24  stopped at:  11
Date:   2022-11-22  stopped at:  6
Date:   2022-11-19  stopped at:  441
Date:   2022-11-09  stopped at:  127
Date:   2022-11-02  stopped at:  46
Date:   2022-11-04  stopped at:  14
Date:   2022-11-18  stopped at:  3
Date:   2022-11-01  stopped at:  20
Date:   2022-11-15  stopped at:  1256
Should have stopped at:  2141
Should have stopped at:  2142
Should have stopped at:  2143
Should have stopped at:  2144
Should have stopped at:  2145
Should have stopped at:  2146
Should have stopped at:  2147
Should have stopped at:  2148
Should have stopped at:  2149
Should h

  0%|          | 0/60166 [00:00<?, ?it/s]

Date:   2022-11-03  stopped at:  180


In [ ]:
sns.lineplot(train_rewards)

In [ ]:
sns.lineplot(train_actions)

In [ ]:
test_actions